# Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Personalize

In [ ]:
# These should be the parent and subfolder names on Google Drive:
exp_label = "your_experiment_name" # e.g., PPP3
exp_spec = "your_experiment_specification" # e.g., pilot2

# This pipeline uses automatic speech-to-text software provided by https://www.assemblyai.com/
# Create an account and replace the code below with your personal API key.
api_key = 'your_api_key'

# Indicate how many files should exist for each participant (unique subject_id).
# The script will remove files from participants that have fewer or more files than this number.
num_files = 'your_number'

# Define the language for transcription - check language_code here: https://www.assemblyai.com/docs/getting-started/supported-languages
language_code = "en_us" # Default is American English

# Install packages

In [ ]:
pip install assemblyai

In [ ]:
pip install praat-textgrids

In [ ]:
pip install pydub

In [ ]:
pip install SpeechRecognition

# Functions & file paths

In [ ]:
# path to folder containing wav files
wav_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/wav'

# get list of wav file names
import os
filenames = [w for w in os.listdir(wav_dir) if (".wav" in w)]

# path to output folder with transcription dictionary
output_dictionary_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/transc_output'

# path to zip folder with wav files for chronset
chronset_input_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/chronset_input'

# path to output folder with onset
chronset_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/chronset_results'

# path to output folder with onset
output_onset_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/onset_output'

# path to output folder with textgrids (with transcriptions)
tg_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_raw'

os.makedirs(output_dictionary_dir, exist_ok=True)
os.makedirs(chronset_input_dir, exist_ok=True)
os.makedirs(chronset_dir, exist_ok=True)
os.makedirs(output_onset_dir, exist_ok=True)

In [ ]:
# transcribe wav files and save transcriptions in a dictionary
def transcribe(api_key, filenames, wav_dir, output_dictionary_dir, language_code):
    import os
    import assemblyai as aai
    import pickle

    # your AssemblyAI API key
    aai.settings.api_key = api_key

    # set language
    config = aai.TranscriptionConfig(language_code=language_code)

    # create transcriber
    transcriber = aai.Transcriber(config=config)

    # initialize transcription dictionary
    transc_dict = {}

    for file in filenames:
        # load wav file
        filepath = os.path.join(wav_dir, file)

        # transcribe wav file
        transcript = transcriber.transcribe(filepath)

        if transcript.error:
            print(file, transcript.error)
            transc_dict[file] = 'not_recognized'
        else:
            # save transcription in dictionary
            transc_dict[file] = transcript.text  or 'not_recognized'
            print(f"{file}: {transcript.text}")

    # save the dictionary
    with open(os.path.join(output_dictionary_dir, "transcriptions.p"), "wb") as f:
        pickle.dump(transc_dict, f)

    # format final transcriptions
    final_transc = {os.path.splitext(key)[0]: val for key, val in transc_dict.items()}

    return final_transc

In [ ]:
# This function is adapted from: https://github.com/Masato-Nakamura-3/TransCloze
def chronset_results_to_dictionary(chronset_dir, output_onset_dir):
    import os
    import pandas as pd
    import pickle

    onset_dict = dict()
    for textf in [t for t in os.listdir(chronset_dir) if ".txt" in t]:
        onset_df = pd.read_table(os.path.join(chronset_dir, textf), names = ["wav", "rt"])
        temp_dict = dict(zip([os.path.splitext(w)[0] for w in onset_df["wav"]], onset_df["rt"] / 1000))
        onset_dict.update(temp_dict)

    # save the dictionary
    with open(os.path.join(output_onset_dir, "onset.p"), "wb") as f:
        pickle.dump(onset_dict, f)

    return onset_dict

In [ ]:
# This function is adapted from: https://github.com/Masato-Nakamura-3/TransCloze
# This function generates .TextGrid files for Praat
# You can optionally pass a dictionary for the onset and/or for the duration of the files
# The onset function is basically for Chronset
# You can also give it context info to help hand correction
# If the audio files have heterogeneous durations, add an argument of "inconsistent = 1"
# The function replaces the RT with 0.1 if the RT is 0 or shorter, or is NaN, producing an error message
def create_textgrids(dict_transcription, output_dir, wav_dir, onset = dict(), inconsistent = False):
    import os
    import textgrids
    import numpy as np
    from pydub import AudioSegment

    filenames = [os.path.splitext(i)[0] for i in os.listdir(wav_dir) if (".wav" in i)]

    for k in filenames:
        # Load the TextGrid template and edit it
        tg = textgrids.TextGrid(f"/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_raw/temp_note.TextGrid")

        # If the duration of the audio files are inconsistent, edit each TextGrid files to match the duration of the audio files
        if inconsistent == True:
            audio = AudioSegment.from_file(os.path.join(wav_dir, k + ".wav"))
            tg["words"][1].xmax = tg["words"][-1].xmin = audio.duration_seconds - 0.001

            if tg.xmax != audio.duration_seconds:
                tg.xmax = audio.duration_seconds

                for tier in tg.keys():
                    tg[tier][-1].xmax = audio.duration_seconds

        tg["words"][1].text = dict_transcription[k + ".wav"]

        if k in onset.keys():
            # If the onset is not detected or is negative, set 0.1
            if np.isnan(onset[k]) or onset[k] <= 0 :
                print("No onset for ", k)
                tg["words"][0].xmax = tg["words"][1].xmin = 0.1
            else:
                tg["words"][0].xmax = tg["words"][1].xmin = onset[k]

        item_id = k.split("_")[1]

        tg.write(os.path.join(output_dir, k + ".TextGrid"))

    print("All textgrid files generated.")

# Step 1. Run automatic transcriptions

In [ ]:
import os
len(os.listdir(wav_dir))

In [ ]:
# List all files in the wav folder
wav_files = os.listdir(wav_dir)

# Dictionary to count occurrences of each unique ID
file_ids = {}

# Iterate through the files in the folder
for filename in wav_files:
    if filename.endswith('.wav'):
        # Extract the unique ID (last four letters after the last '_')
        unique_id = filename.split('_')[-1].split('.')[0]

        # Count the occurrence of each unique ID
        if unique_id in file_ids:
            file_ids[unique_id] += 1
        else:
            file_ids[unique_id] = 1

# Classify the IDs based on whether they have exactly 80 files
ids_with_num_files = [id_ for id_, count in file_ids.items() if count == num_files]
ids_with_fewer_or_more_files = [id_ for id_, count in file_ids.items() if count != num_files]

# Print the results
print(f"IDs with exactly {num_files} files:")
print(len(ids_with_num_files))
print(ids_with_num_files)

print(f"\nIDs with fewer or more than {num_files} files:")
print(len(ids_with_fewer_or_more_files))
print(ids_with_fewer_or_more_files)

In [ ]:
# Remove files with IDs that have fewer or more than num_files
for filename in wav_files:
    if filename.endswith('.wav'):
        # Extract the unique ID (last four letters after the last '_')
        unique_id = filename.split('_')[-1].split('.')[0]
        # If the ID has fewer or more than num_files, delete the file
        if unique_id in ids_with_fewer_or_more_files:
            file_path = os.path.join(wav_dir, filename)
            os.remove(file_path)
            print(f"Deleted: {filename}")
print(f"Files with IDs that have fewer or more than {num_files} files have been deleted.")

In [ ]:
len(os.listdir(wav_dir))

In [ ]:
# get list of wav file names
import os
filenames = [w for w in os.listdir(wav_dir) if (".wav" in w)]

# generate transcriptions and store them in a dictionary
dict_transcription = transcribe(api_key, filenames, wav_dir, output_dictionary_dir, language_code)

In [ ]:
len(dict_transcription)

# Step 2. Get timing measures - select one of the two options below

# Option A. Get speech onsets (uses Chronset)

This section uses Chronset to get onset data and combines with with transcriptions

In [ ]:
import os
import zipfile
from pydub import AudioSegment
import speech_recognition as sr

# Set the name of the zip files
# The output files will be named as "{experiment_name}_{number}".zip (e.g. planet_cloze_1.zip)
zipname = f"{exp_label}_{exp_spec}_"

# # Check wav files for errors
# for f in filenames:
#     fsize = os.path.getsize(os.path.join(wav_dir, f))
#     if fsize < 100000:
#         print(f)
#         print(fsize)
#         with sr.AudioFile(os.path.join(wav_dir, f)) as wavf:
#             print(wavf.DURATION)

# Create batches of 500 wav files (i.e., maximum file counts Chronset can handle)
# Make lists of 500 or fewer wav files
fewerthan500 = []
temp = []
for f in filenames:
    temp.append(f)
    if len(temp) == 500:
        fewerthan500.append(temp)
        temp = []
if len(temp) > 0:
    fewerthan500.append(temp)

# Total nuber of wav files
len(filenames)
# Number of zip files
len(fewerthan500)

# Output zip files (with 500 or fewer wav files)
fnum = 0
for filelist in fewerthan500:
    fnum = fnum + 1
    with zipfile.ZipFile(os.path.join(chronset_input_dir, zipname+str(fnum) + ".zip"), 'w', compression=zipfile.ZIP_DEFLATED) as new_zip:
        for f in filelist:
            new_zip.write(os.path.join(wav_dir, f), arcname = f)

Go to https://www.bcbl.eu/databases/chronset and upload each zip file that is in the **chronset_input** folder.

Retrieve Chronset results from your mailbox and upload those files to the **chronset_results** folder.

In [ ]:
# get onsets and store them in a dictionary
import pickle
onset = chronset_results_to_dictionary(chronset_dir, output_onset_dir)
onset = pickle.load( open( f"/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/onset_output/onset.p", "rb" ) )

In [ ]:
# generate textgrid files and store them in tg_dir
import pickle
transcriptions = pickle.load( open( f"/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/transc_output/transcriptions.p", "rb" ) )
create_textgrids(transcriptions, tg_dir, wav_dir, onset, inconsistent = True)

In [ ]:
# generate results
import os
import io
import textgrids
import pandas as pd
import numpy as np

tg_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_raw'

results = []
for tg_loc in [f for f in os.listdir(tg_dir) if f.endswith(".TextGrid")]:
    tg_path = os.path.join(tg_dir, tg_loc)
    filename = tg_loc.rsplit(".", 1)[0]
    item = filename.split("_")[0]
    item_type = "_".join(filename.split("_")[1:-1])
    subject_id = filename.split("_")[-1]

    try:
        tg = textgrids.TextGrid(tg_path)
        for word in tg['words']:
            if word.text.strip():
                results.append([filename, item, item_type, subject_id, word.text, word.xmin, word.xmax, word.dur])
    except Exception as e:
        print(f"Skipping file {tg_loc} due to error: {e}")
        # Insert a row indicating the file was not recognized due to an error
        results.append([filename, item, item_type, subject_id, "not_recognized", None, None, None])

df = pd.DataFrame(results, columns = ['filename', 'item', 'item_type', 'subject_id', 'word', 'rt_onset', 'rt_offset', 'rt_dur'])
df[['type', 'condition']] = pd.DataFrame(df['item_type'].apply(lambda x: x.split('_') if '_' in x else [x, None]).tolist(), index=df.index)
df['word'] = df['word'].apply(lambda x: x.strip().lower())
df['item'] = df['item'].astype(int)
df['word_order'] = df.groupby('filename')['word'].cumcount() + 1

df = df.reset_index(drop=True)
df

# Option B. Get speech onsets + alignments in multi-word utterances (uses MFA)

In [ ]:
# generate textgrid files and store them in tg_dir
import pickle
transcriptions = pickle.load( open( f"/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/transc_output/transcriptions.p", "rb" ) )
create_textgrids(transcriptions, tg_dir, wav_dir, inconsistent = True)

In [ ]:
len(transcriptions)

In [ ]:
import os
len(os.listdir(tg_dir))

## Error analysis before running MFA

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
len(os.listdir(tg_dir))

In [ ]:
import os
import io
import textgrids
import pandas as pd
import numpy as np

tg_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_raw'

# generate results df from textgrid files
results = []
for tg_loc in [f for f in os.listdir(tg_dir) if ((f.endswith(".TextGrid")) & (f != 'temp_note.TextGrid'))]:
    tg_path = os.path.join(tg_dir, tg_loc)
    filename = tg_loc.rsplit(".", 1)[0]
    item = filename.split("_")[0]
    item_type = "_".join(filename.split("_")[1:-1])
    subject_id = filename.split("_")[-1]
    try:
        tg = textgrids.TextGrid(tg_path)
        results.append([filename, item, item_type, subject_id, tg['words'][1].text.strip()])
    except Exception as e:
        print(f"Skipping file {tg_loc} due to error: {e}")
        results.append([filename, item, item_type, subject_id, "not_recognized"])
df = pd.DataFrame(results, columns = ['filename', 'item', 'item_type', 'subject_id', 'response'])
df[['type', 'condition']] = pd.DataFrame(df['item_type'].apply(lambda x: x.split('_') if '_' in x else [x, None]).tolist(), index=df.index)
df['response'] = df['response'].apply(lambda x: x.strip().lower())
df['item'] = df['item'].astype(int)
df = df.reset_index(drop=True)
df.response = df.response.str.strip('.')

# add subject id from PCIbex results file
PCIbex_results = pd.read_csv(f'/content/drive/MyDrive/{exp_label}/{exp_spec}/{exp_label}_PCIbex_results_{exp_spec}.csv')
PCIbex_results.rename(columns = {'#':'time_id', 'Unnamed: 5':'condition_general', 'Unnamed: 7':'element_type', 'Unnamed: 10':'filename', 'Unnamed: 12':'group', 'Unnamed: 13':'subject_id', 'Unnamed: 14':'item_num', 'Unnamed: 15':'condition','Unnamed: 16':'response'}, inplace = True)
PCIbex_results_MedRec = PCIbex_results[PCIbex_results["element_type"].str.contains("MediaRecorder") == True]
PCIbex_results_id = PCIbex_results_MedRec[["time_id","subject_id"]]
df_results_id = pd.merge(df, PCIbex_results_id[['time_id','subject_id']].drop_duplicates(), how='right', on=['subject_id'])
df = df_results_id.dropna(subset=['filename'])

# # add attention check column after checking
# df['attention_check'] = 1

In [ ]:
# save results file
df.to_csv(f'/content/drive/MyDrive/{exp_label}/{exp_spec}/df_raw.csv', index=False)

In [ ]:
# Example: select critical trials only

# read results file
import pandas as pd
df = pd.read_csv(f'/content/drive/MyDrive/{exp_label}/{exp_spec}/df_raw.csv')

name_of_stim_file = 'your_stim_file_name'
name_of_critical_results_file = 'your_critical_results_file_name'

# df
df_critical = df[df.item < 25]
df_critical = df_critical.rename(columns={"type":"possessor_gender",'condition':'gender_match'})
# stim
stim_critical = pd.read_csv(f'/content/drive/MyDrive/{exp_label}/{exp_spec}/{name_of_stim_file}.csv')
stim_critical = stim_critical.drop(['probe','wav_filename'], axis = 1)
stim_critical = stim_critical.rename(columns={'sentence_target':'target_response','voice':'sentence_type','condition':'gender_match','group':'list'})
stim_critical['target_response'] = stim_critical['target_response'].apply(lambda x: x.strip())
stim_critical['target_response'] = stim_critical['target_response'].apply(lambda x: x.strip('.'))
stim_critical['target_response'] = stim_critical['target_response'].apply(lambda x: x.lower())
stim_critical['possessor'] = stim_critical['possessor'].str.lower()
stim_critical["condition"] = stim_critical["sentence_type"] + "_" + stim_critical["gender_match"]
stim_critical["target_response"] = stim_critical["target_response"].apply(lambda x: ' '.join(x.split()[:5]))
# combine df and stim
df_stim_critical = pd.merge(df_critical,stim_critical,how='left',on=['item','possessor_gender','gender_match']).drop_duplicates()
df_stim_critical = df_stim_critical[['filename', 'item', 'subject_id',
                                     'possessor_gender', 'gender_match', 'response','sentence_type',
                                     'possessor', 'adverb', 'verb', 'pronoun', 'possessee',
                                     'sentence', 'target_response','answer_target', 'possessee_type', 'subject_animacy', 'speaker_image',
                                     'list','condition','attention_check']]
# mark errors
# error = 0 IF response exactly matches first five words of full target sentence - for RT analysis
# exclude = 0 IF response includes verb and doesn't include first person
df_stim_critical['error'] = df_stim_critical.apply(lambda row: 0 if (str(row['target_response']) in str(row['response'])) else 1, axis=1)
df_stim_critical.loc[df_stim_critical.error == 0, 'response_type'] = 'target'
df_stim_critical.loc[df_stim_critical.response == 'not_recognized', 'response_type'] = 'no_response'
df_stim_critical.loc[df_stim_critical.response == ' ', 'response_type'] = 'no_response'
df_stim_critical.loc[df_stim_critical.response == df_stim_critical.target_response, 'response_type'] = 'target'
df_stim_critical['exclude'] = df_stim_critical.apply(lambda row: 0 if (str(row['verb']) in str(row['response']) and not any(x in str(row['response']) for x in [' i ', ' me ', ' my '])) else 1, axis=1)
df_stim_critical.loc[df_stim_critical.exclude == 1, 'response_type'] = 'exclude'

df_stim_critical.to_csv(f'/content/drive/MyDrive/{exp_label}/{exp_spec}/{name_of_critical_results_file}.csv', index=False)
df_stim_critical.head()

## MFA

### Prepare MFA input files

In [ ]:
# read cleaned df
import pandas as pd
name_of_critical_results_file = 'your_name_of_critical_results_file'
df_clean = pd.read_csv(f'/content/drive/MyDrive/{exp_label}/{exp_spec}/{name_of_critical_results_file}.csv').reset_index(drop=True)
# select only target responses for RT analysis
df_corr = df_clean[df_clean.error != 1]
# retrieve filenames
df_corr_filenames = df_corr.filename.unique().tolist()

# add more filenames if necessary
name_of_control_results_file = 'your_name_of_control_results_file'
df_clean2 = pd.read_csv(f'/content/drive/MyDrive/{exp_label}/{exp_spec}/{your_name_of_control_results_file}.csv').reset_index(drop=True)
# select only target responses for RT analysis
df_corr2 = df_clean2[df_clean2.error != 1]
# retrieve filenames
df_corr_filenames2 = df_corr2.filename.unique().tolist()

# combine
df_corr_filenames.extend(df_corr_filenames2)
len(df_corr_filenames)

In [ ]:
# copy matching files from wav and tg_raw folders
import os
import shutil
wav_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/wav'
tg_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_raw'
mfa_input_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/MFA_input_raw'
os.makedirs(mfa_input_dir, exist_ok=True)
for filename in df_corr_filenames:
    # Define full paths for each file in the wav and tg_raw folders
    wav_file_path = os.path.join(wav_dir, f'{filename}.wav')
    tg_file_path = os.path.join(tg_dir, f'{filename}.TextGrid')
    # Copy matching .wav files
    if os.path.isfile(wav_file_path):
        shutil.copy(wav_file_path, mfa_input_dir)
    # Copy matching .TextGrid files
    if os.path.isfile(tg_file_path):
        shutil.copy(tg_file_path, mfa_input_dir)
print("Matching files have been copied to the MFA_input_raw folder.")

In [ ]:
mfa_input_files_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/MFA_input_raw'
len(os.listdir(mfa_input_files_dir))

In [ ]:
# rename filenames for MFA
import os
import re
mfa_input_files_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/MFA_input_raw'
for root, dirs, files in os.walk(mfa_input_files_dir):
    for filename in files:
        # Process only .wav and .TextGrid files
        if filename.endswith('.wav') or filename.endswith('.TextGrid'):
            # Split the filename by underscores and the extension
            parts = re.split(r'[_\.]', filename)
            if len(parts) == 5:  # change depending on number of parts in filenames, including extension
                # Rearrange the filename with the ID at the end
                new_filename = f"{parts[3]}_{parts[0]}_{parts[1]}_{parts[2]}.{parts[4]}"
                # Full path to the original and new filenames
                old_file = os.path.join(root, filename)
                new_file = os.path.join(root, new_filename)
                # Rename the file
                os.rename(old_file, new_file)
                print(f"Renamed: {filename} to {new_filename}")

In [ ]:
# duplicate folder
import os
import shutil
mfa_input_files_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/MFA_input_raw'
mfa_input_folders_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/MFA_input_subfolders'
if os.path.exists(mfa_input_folders_dir):
    print(f"The directory '{mfa_input_folders_dir}' already exists. Please remove it or choose a different name.")
else:
    shutil.copytree(mfa_input_files_dir, mfa_input_folders_dir)
    print(f"Folder '{mfa_input_files_dir}' has been successfully duplicated to '{mfa_input_folders_dir}'.")

In [ ]:
mfa_input_folders_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/MFA_input_subfolders'
print(len(os.listdir(mfa_input_files_dir)))
print(len(os.listdir(mfa_input_folders_dir)))

In [ ]:
# divide into smaller batches for MFA processing
import os
from shutil import move
files_per_subfolder = 500  # Adjust the number of pairs you want per subfolder
all_files = os.listdir(mfa_input_folders_dir)
wav_files = sorted([f for f in all_files if f.endswith('.wav')])
textgrid_files = sorted([f for f in all_files if f.endswith('.TextGrid')])
pairs = []
for wav_file in wav_files:
    base_name = os.path.splitext(wav_file)[0]
    textgrid_file = base_name + '.TextGrid'
    if textgrid_file in textgrid_files:
        pairs.append((wav_file, textgrid_file))

mfa_input_folders_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/MFA_input_subfolders'
def create_subfolders_and_move_files(pairs, source_directory, files_per_subfolder):
    subfolder_count = 1
    for i in range(0, len(pairs), files_per_subfolder):
        # Create a new subfolder in the source directory
        subfolder_path = os.path.join(source_directory, f'subfolder_{subfolder_count}')
        os.makedirs(subfolder_path, exist_ok=True)
        # Move a batch of pairs to the new subfolder
        for wav_file, textgrid_file in pairs[i:i + files_per_subfolder]:
            move(os.path.join(source_directory, wav_file), os.path.join(subfolder_path, wav_file))
            move(os.path.join(source_directory, textgrid_file), os.path.join(subfolder_path, textgrid_file))
        subfolder_count += 1
    print("Files have been successfully divided into subfolders.")

create_subfolders_and_move_files(pairs, mfa_input_folders_dir, files_per_subfolder)

In [ ]:
# count number of files and folders
mfa_input_folders_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/MFA_input_subfolders'
def count_files_in_directory(directory):
    file_count = 0
    for root, dirs, files in os.walk(directory):
        file_count += len(files)
    return file_count
file_count = count_files_in_directory(mfa_input_folders_dir)
print(f"Total number of folders: {len(os.listdir(mfa_input_folders_dir))}")
print(f"Total number of files: {file_count}")

In [ ]:
import shutil
from google.colab import files

# Define the folder path
mfa_input_folders_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/MFA_input_subfolders'
zip_path = '/content/MFA_input_subfolders.zip'

# Zip the folder
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', mfa_input_folders_dir)

# Download the zip file
files.download(zip_path)

### Run MFA locally

Refresh Google Drive and wait until you see all files in subfolders in the **MFA_input_subfolders** folder.

On your local computer inside MFA folder (e.g., ~/Documents/MFA):
*   Create new folder and use same file name format as in Google Drive (e.g., **PPP3_pilot2**)
*   Upload **run_mfa_on_subsets.sh** file (from Github pipeline)

Inside the experiment folder (e.g., ~/Documents/MFA/PPP3_pilot2):
*   Upload **MFA_input_subfolders** folder (from Google Drive)
*   Create new folder **MFA_output_subfolders**

Open **run_mfa_on_subsets.sh** file with text editor:
*   Change file paths to your folders, save and close

Open terminal and run each code below line by line:
*   conda activate aligner
*   cd ~/Documents/MFA
*   chmod +x run_mfa_on_subsets.sh
*   ./run_mfa_on_subsets.sh

Once you finish, deactivate the environment:
*   conda deactivate

Upload the **MFA_output_subfolders** folder to your experiment folder on Google Drive

## Combine files after running MFA

Wait until all files have successfully been uploaded to **MFA_output_subfolders** folder

In [ ]:
# check how many tg files
import os
mfa_input_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/MFA_input_subfolders'
mfa_output_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/MFA_output_subfolders'
def count_textgrid_files_in_directory(directory):
    textgrid_count = 0
    for root, dirs, files in os.walk(directory):
        textgrid_count += sum(1 for file in files if file.endswith('.TextGrid'))
    return textgrid_count
print(f"Total number of tg files in MFA input directory: {count_textgrid_files_in_directory(mfa_input_dir)}")
print(f"Total number of tg files in MFA output directory: {count_textgrid_files_in_directory(mfa_output_dir)}")
print(f"Total number of tg files not MFA-processed: {count_textgrid_files_in_directory(mfa_input_dir)/2 - count_textgrid_files_in_directory(mfa_output_dir)}")

In [ ]:
# copy only TextGrid files
mfa_output_tg_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/MFA_output_tg'
os.makedirs(mfa_output_tg_dir, exist_ok=True)
def copy_textgrid_files(source_dir, dest_dir):
    from shutil import copy2
    for root, dirs, files in os.walk(source_dir):
        for file in files:
            if file.endswith('.TextGrid'):
                source_file_path = os.path.join(root, file)
                destination_file_path = os.path.join(dest_dir, file)
                # Check if a file with the same name already exists in the destination directory
                if os.path.exists(destination_file_path):
                    # If so, create a new file name to avoid overwriting
                    base, extension = os.path.splitext(file)
                    counter = 1
                    new_file_name = f"{base}_{counter}{extension}"
                    destination_file_path = os.path.join(dest_dir, new_file_name)
                    # Increment the counter until a unique name is found
                    while os.path.exists(destination_file_path):
                        counter += 1
                        new_file_name = f"{base}_{counter}{extension}"
                        destination_file_path = os.path.join(dest_dir, new_file_name)
                # Copy the file
                copy2(source_file_path, destination_file_path)
    print("All TextGrid files have been copied to the folder: MFA_output_tg")
copy_textgrid_files(mfa_output_dir, mfa_output_tg_dir)
print(f"Total number of textgrid files in MFA_output_tg folder: {len(os.listdir(mfa_output_tg_dir))}")

In [ ]:
# rename filenames back into original filenames
import os
import re
mfa_output_tg_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/MFA_output_tg'
for root, dirs, files in os.walk(mfa_output_tg_dir):
    for filename in files:
        # Process only .wav and .TextGrid files
        if filename.endswith('.wav') or filename.endswith('.TextGrid'):
            # Split the filename by underscores and the extension
            parts = re.split(r'[_\.]', filename)
            if len(parts) == 5:  # change depending on number of parts in filenames, including extension
                # Rearrange the filename with the ID at the end
                new_filename = f"{parts[1]}_{parts[2]}_{parts[3]}_{parts[0]}.{parts[4]}"
                # Full path to the original and new filenames
                old_file = os.path.join(root, filename)
                new_file = os.path.join(root, new_filename)
                # Rename the file
                os.rename(old_file, new_file)
                print(f"Renamed: {filename} to {new_filename}")

In [ ]:
# check number of tg files in pre-MFA & post-MFA folders
mfa_input_tg_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/MFA_input_tg'
mfa_output_tg_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/MFA_output_tg'
len(os.listdir(mfa_input_tg_dir)), len(os.listdir(mfa_output_tg_dir))

In [ ]:
# generate results
import os
import io
import textgrids
import pandas as pd
import numpy as np

tg_MFA_processed_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_MFA_processed'

results = []
for tg_loc in [f for f in os.listdir(tg_MFA_processed_dir) if f.endswith(".TextGrid")]:
    tg_path = os.path.join(tg_MFA_processed_dir, tg_loc)
    filename = tg_loc.rsplit(".", 1)[0]
    item = filename.split("_")[0]
    item_type = "_".join(filename.split("_")[1:-1])
    subject_id = filename.split("_")[-1]

    try:
        tg = textgrids.TextGrid(tg_path)
        for word in tg['words']:
            if word.text.strip():
                results.append([filename, item, item_type, subject_id, word.text, word.xmin, word.xmax, word.dur])
    except Exception as e:
        print(f"Skipping file {tg_loc} due to error: {e}")
        # Insert a row indicating the file was not recognized due to an error
        results.append([filename, item, item_type, subject_id, "not_recognized", None, None, None])

df = pd.DataFrame(results, columns = ['filename', 'item', 'item_type', 'subject_id', 'word', 'rt_onset', 'rt_offset', 'rt_dur'])
df[['type', 'condition']] = pd.DataFrame(df['item_type'].apply(lambda x: x.split('_') if '_' in x else [x, None]).tolist(), index=df.index)
df['word'] = df['word'].apply(lambda x: x.strip().lower())
df['item'] = df['item'].astype(int)
df['word_order'] = df.groupby('filename')['word'].cumcount() + 1

df = df.reset_index(drop=True)
df

In [ ]:
df.subject_id.nunique()